In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import math
import time
from datetime import datetime
import pickle
from pathlib import Path
import os
import requests
from bs4 import BeautifulSoup
import lxml
import traceback
import argparse
import pandas as pd
import re
from html import unescape


In [ ]:
path_main = 'U:/01 Elbrus Bootcamp/56 W11D3 Parsing/Project'
path_data = path_main + '/data'
path_images = path_main + '/images'


In [ ]:
data_file = 'books.csv'
data = pd.DataFrame([], columns=['page_url', 'image_url', 'author', 'title', 'annotation', 'book_image_file_name'])

In [ ]:
# data.to_csv(path_main + '/data/books.csv')

In [ ]:
# подготовка  к запросам
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) " 
           "AppleWebKit/537.36 (KHTML, like Gecko) "
           "Chrome/39.0.2171.95 Safari/537.36"
           }

In [ ]:
url =  'https://www.litres.ru/book/anne-dar/pobedonosec-71641255/'
#[page_url]

In [ ]:
r = requests.get(url, headers=headers, timeout=60)


In [ ]:
r.status_code # если 200 - то ок

In [ ]:
print(r.text)

In [ ]:
# if r.status_code == 200:
#     # если все ок
#     soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
soup1 = BeautifulSoup(r.text, 'html.parser')

In [ ]:
title = soup1.find('body').find('div', id='layout-root').find('main', id='main').find('div', id='book-card__wrapper').find('h1', itemprop='name')
# title = soup1.find('body').find('div', id='layout-root').find('main', id='main').find('h1', itemprop='name')
print(title)

In [ ]:
book_title = title.getText() #[title]
book_title

In [ ]:
author = soup1.find('body').find('div', id='layout-root').find('main', id='main').find('div', class_='Truncate_truncated__jKdVt')
author.getText() #[author]

In [ ]:
image_link = soup1.find('body').find('div', id='layout-root').find('main', id='main').find('img', itemprop='image')['src']
image_link # [image_url]
# пока сохроаним только ссылку на картинку

In [ ]:
# подготовка  к запросам
headers3 = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) " 
           "AppleWebKit/537.36 (KHTML, like Gecko) "
           "Chrome/39.0.2171.95 Safari/537.36",
           "Referer": "https://www.litres.ru/book/anne-dar/pobedonosec-71641255/"
           }

In [ ]:
url3 = "https://api.litres.ru/tracker/api/event/"

In [ ]:
r3 = requests.get(url, headers=headers3, timeout=60)

In [ ]:
r3.status_code

In [ ]:
print(r3.text)

In [ ]:
soup3 = BeautifulSoup(r3.text, 'html.parser')

In [ ]:
content = soup3.find('body').find('div', id='layout-root').find('main', id='main').find('div', class_='BookCard_book__content__7J8Fc').find('div', class_='BookCard_truncate__jrVwM')
content

In [ ]:
book_content = content.getText() #[title]
book_content

In [ ]:
# пробуем добыть каталог ---------------------------------

In [ ]:
catalogue_url =  'https://www.litres.ru/pages/new_genres/'

In [ ]:
r_catalogue = requests.get(catalogue_url, headers=headers, timeout=60)

In [ ]:
print(r_catalogue.text)

In [ ]:
catalogue_soup1 = BeautifulSoup(r_catalogue.text, 'html.parser')

In [ ]:
#content = soup3.find('body').find('div', id='layout-root').find('main', id='main').find('div', class_='BookCard_book__content__7J8Fc').find('div', class_='BookCard_truncate__jrVwM')
content = catalogue_soup1.find_all('a', tabindex="0")
content

In [ ]:
total_catalogue_dict = {'Group':[], 'Group_link':[]}

a1 = str(content).split(sep="</a>, ")
for line in range(0, len(a1)):    
    if a1[line].find('StyledLink_link__UWNRS Item_item__LNZxG') > 0:
        a1_bs = BeautifulSoup(a1[line], 'html.parser')
        group_link = a1_bs.find('a')['href']
        group = a1_bs.getText()
        total_catalogue_dict['Group'].append(group)
        total_catalogue_dict['Group_link'].append(group_link)

total_catalogue_dict = pd.DataFrame.from_dict(total_catalogue_dict)
total_catalogue_dict

In [ ]:
total_catalogue_dict.to_csv('data/book_groups.csv')

In [ ]:
# https://www.litres.ru/genre/knigi-detektivy-5022/?view=showroom
# /genre/knigi-detektivy-5022/

In [ ]:
# начинаем собирать ссылки на книги с одной страницы  ---------------------------------

In [ ]:
new_link_portion = total_catalogue_dict.iloc[1,1]
base_link = 'https://www.litres.ru' + new_link_portion + '?view=showroom&page=2' # !!!!!!!!!!!!!!!!!!! page
base_link


In [ ]:
single_page_r = requests.get(base_link, headers=headers, timeout=60)

In [ ]:
single_page_r.status_code # если 200 - то ок

In [ ]:
single_page_soup = BeautifulSoup(single_page_r.text, 'html.parser')

In [ ]:
title = single_page_soup.find('div', class_='PaginatedContent_wrapper___IXwB').find('div', class_='PaginatedContent_content__HG6bS').find_all('a', tabindex='0')
title

In [ ]:
total_page_link = {'Name':[], 'Link':[]}

a1 = str(title).split(sep="</a>, ")
for line in range(0, len(a1)):    
    if a1[line].find('aria-label') > 0:
        a1_bs = BeautifulSoup(a1[line], 'html.parser')
        name_link = a1_bs.find('a')['href'] 
        name = a1_bs.find('a')['aria-label'] 
        total_page_link['Name'].append(name)
        total_page_link['Link'].append(name_link)

total_page_link = pd.DataFrame.from_dict(total_page_link)
total_page_link

In [ ]:
# https://www.litres.ru/book/stiv-kavana/pyatdesyat-na-pyatdesyat-71516344/
# /book/anne-dar/pobedonosec-71641255/

In [ ]:
# теперь чилсо страниц в группе книг ------------------------------

In [ ]:
#беру предыдущий запрос к странице
total_pages = single_page_soup.find('div', class_='PaginatedContent_paginator___2ugs').find_all('li', class_='Paginator_page___fg9G')
b1 = BeautifulSoup(str(total_pages).split(sep=",")[-1], 'html.parser').getText()
non_decimal = re.compile(r'[^\d.]+')
b1 = non_decimal.sub('', b1)
int(b1)
